In [1]:
import time
import random
import math

### Functions

In [2]:
# Lembrando que os indices começam com 0
# [1,4, 3,2, 7,3, 6,3, 2,4, 5,3]
def imprimir_agenda(pessoas, agenda):

    voos = get_data()
    
    idx_voo = -1

    for i in range(len(agenda)//2):
        nome   = pessoas[i][0]
        origem = pessoas[i][1]

        idx_voo += 1

        ida = voos[(origem, destino)][agenda[idx_voo]]

        idx_voo +=1

        volta = voos[(destino, origem)][agenda[idx_voo]]

        print(f'{nome : <8}  : {origem}/{destino} - {ida[0]:<7} {ida[1]:<7} R$ {ida[2]:<5} | {destino:>5}/{origem} - {volta[0]: <7} {volta[1]:<7} R$ {volta[2]}')



def get_minutes(hora):
    _struct_time = time.strptime(hora, '%H:%M')
    minutos = _struct_time[3] * 60 + _struct_time[4]

    return minutos


def get_data():

    voos = {}

    for linha in open('voos_hm.txt'):
    # print(linha)
        _origem, _destino, _saida, _chegada, _preco = linha.split(',')
        voos.setdefault((_origem, _destino), [])
        voos[(_origem, _destino)].append((_saida, _chegada, int(_preco)))

    return voos

In [3]:
voos = get_data()

In [4]:
voos[('CNF', 'GYN')]

[('6:19', '8:13', 239),
 ('8:04', '10:59', 136),
 ('9:31', '11:43', 210),
 ('11:07', '13:24', 171),
 ('12:31', '14:02', 234),
 ('14:05', '15:47', 226),
 ('15:07', '17:21', 129),
 ('16:35', '18:56', 144),
 ('18:25', '20:34', 205),
 ('20:05', '21:44', 172)]

In [5]:
pessoas = [('Kevin', 'SLZ'),
           ('Joao', 'VCP'),
           ('Dani', 'POA'),
           ('Rafa', 'FOR'),
           ('Mari', 'GRU'),
           ('Isa', 'GYN')]

destino = 'CNF'

In [6]:
agenda = [1,4, 3,2, 7,3, 6,3, 2,4, 5,3]
imprimir_agenda(pessoas, agenda)

Kevin     : SLZ/CNF - 8:04    10:11   R$ 95    |   CNF/SLZ - 12:08   14:05   R$ 142
Joao      : VCP/CNF - 10:30   14:57   R$ 290   |   CNF/VCP - 9:49    13:51   R$ 229
Dani      : POA/CNF - 17:08   19:08   R$ 262   |   CNF/POA - 10:32   13:16   R$ 139
Rafa      : FOR/CNF - 15:34   18:11   R$ 326   |   CNF/FOR - 11:08   14:38   R$ 262
Mari      : GRU/CNF - 9:42    11:32   R$ 169   |   CNF/GRU - 12:08   14:47   R$ 231
Isa       : GYN/CNF - 13:37   15:08   R$ 250   |   CNF/GYN - 11:07   13:24   R$ 171


## Optimization Functions

### 1 - My optimization Function

In [7]:
# Minimize wait time
# Minimize ticket prices

def fn_loss(solution):
    preco_total      = 0 #
    ultima_chegada   = 0 # Pessoa que chega mais tarde no aeroporto
    primeira_partida = get_minutes('23:59') # Pessoa que vai sair mais cedo

    _w_p = 11
    _w_t = 1

    voos = get_data()

    idx_voo = -1

    for i in range(len(solution) // 2):
        origem = pessoas[i][1]

        idx_voo += 1

        ida = voos[(origem, destino)][solution[idx_voo]]

        idx_voo +=1

        volta = voos[(destino, origem)][solution[idx_voo]]

        preco_total += ida[2]
        preco_total += volta[2]

        # Com base nesses valors vamos calcular o tempo que uma
        # pessoa fica esperando no aeroporto
        if ultima_chegada < get_minutes(ida[1]):
            ultima_chegada = get_minutes(ida[1])

        if primeira_partida > get_minutes(volta[0]):
            primeira_partida = get_minutes(volta[0])


    total_espera = 0
    idx_voo      = -1

    for i in range(len(solution) // 2):
        origem = pessoas[i][1]

        idx_voo += 1

        ida = voos[(origem, destino)][solution[idx_voo]]

        idx_voo +=1

        volta = voos[(destino, origem)][solution[idx_voo]]

        total_espera += ultima_chegada - get_minutes(ida[1])
        total_espera += get_minutes(volta[0]) - primeira_partida

    # Penalidade
    if ultima_chegada > primeira_partida:
        preco_total += 50

    return (preco_total * _w_p) + (total_espera * _w_t)



fn_loss(agenda)


30795

### **2 - Random Search (Busca Randômica ou Busca Aleatória)**

- Pega aleatoriamente algumas soluções possiveis do espaço de busca e faz a comparação com o custo
- Para pouquissimos dados pode fornecer resultados de certa forma interessantes
- Aquele com menor custo, será a solução final
- A Pesquisa Aleatoria é comumente utilizada como Benchmark para outros algoritmos

Contras:

- Não utiliza as soluçoes boas que foram encontradas
- Uma solução com baixo custo é semelhante a outra solução com baixo custo
- Pesquisa aleatoria "salta de um lado para o outro"
- Não armazena o resultado das soluçãoes otimas encontradas anteriormente

In [8]:
# Adaptation of optimization algorithms to problem domains by transfer learning
# https://ieeexplore.ieee.org/document/8279737

# Dominio são os valores mínimos e máximos permitidos
dominio = [(0,9)] * (len(pessoas) * 2)
dominio

[(0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9)]

In [9]:
_solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
_solucao


[7, 7, 9, 6, 4, 7, 8, 3, 2, 5, 0, 6]

In [10]:
def random_search(dominio, fn_custo):
    
    _melhor_custo = 9999999999

    # 1 - Primeiro eu faço uma busca aleatoria nos voos.
    for i in range(10000):
        _solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]

        # 2 - Após escolher um voo aleatoriamente, tentamos otimizar o custo com a função custo.
        _custo =  fn_custo(_solucao)

        # 3 - Atualizamos sempre com o melhor custo encontrado e com base no melhor custo pegamos a melhor solução.
        if _custo < _melhor_custo:
            _melhor_custo = _custo
            _melhor_solucao = _solucao


    return _melhor_solucao

In [11]:
%%time
dominio = [(0,9)] * (len(pessoas) * 2)
print(len(dominio))
rand_schedule_solution = random_search(dominio, fn_loss)
rand_schedule_solution

12
CPU times: total: 4.3 s
Wall time: 4.29 s


[9, 6, 0, 3, 5, 5, 9, 4, 5, 3, 4, 3]

In [12]:
random_loss =  fn_loss(rand_schedule_solution)
random_loss

23726

In [13]:
imprimir_agenda(pessoas, rand_schedule_solution)

Kevin     : SLZ/CNF - 20:17   22:22   R$ 102   |   CNF/SLZ - 15:25   16:58   R$ 62
Joao      : VCP/CNF - 6:12    10:22   R$ 230   |   CNF/VCP - 10:51   14:16   R$ 256
Dani      : POA/CNF - 13:40   15:38   R$ 137   |   CNF/POA - 13:37   15:33   R$ 142
Rafa      : FOR/CNF - 19:53   22:21   R$ 173   |   CNF/FOR - 12:37   15:05   R$ 170
Mari      : GRU/CNF - 14:22   16:32   R$ 126   |   CNF/GRU - 10:33   13:11   R$ 132
Isa       : GYN/CNF - 12:18   14:56   R$ 172   |   CNF/GYN - 11:07   13:24   R$ 171


### **2 - Hill Climb (Subida da Enconsta)**

- Começa com uma solução aleatoria inicialmente e procura os melhores vizinhos
- Caminhar em direção a um ponto com maior inclinação ou menor inclinação (a depender do problema)
- Vai rodar ate um ponto que não tem mais para onde descer ou subir (a depender do problema)
- Pode ser utilizado para problemas de Maximização ou Minimmização 
- NÂO é a mesma coisa que o Gradiente Descendente
    - A descida do Gradiente Descendente está mais relacionado a calcular as "direções" para atualização dos valores

https://iaexpert.com.br/index.php/2016/10/25/review-de-livro-programando-a-inteligencia-coletiva/

papers originais das buscas

https://www.researchgate.net/publication/6026283_Optimization_by_Simulated_Annealing

http://www.cs.cornell.edu/selman/papers/pdf/02.encycl-hillclimbing.pdf

In [18]:
def hill_climb(dominio, fn_loss):
     
    _solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]

    while True:

        _neighbors = []

        for i in range(len(dominio)):
            
            if _solucao[i] > dominio[i][0]:
                if _solucao[i] != dominio[i][1]:
                    _neighbors.append(_solucao[0:i] + [_solucao[i] + 1] + _solucao[i + 1:])
            
            if _solucao[i] < dominio[i][1]:
                if _solucao[i] != dominio[i][0]:
                    _neighbors.append(_solucao[0:i] + [_solucao[i] - 1] + _solucao[i + 1:])

        curr = fn_loss(_solucao)
        best = curr

        for i in range(len(_neighbors[i])):
            cost = fn_loss(_neighbors[i])

            if cost < best:
                best =cost
                _solucao = _neighbors[i]
        
        if best == curr:    
            break
    
    return _solucao

In [24]:
%%time

solution_hill_climb = hill_climb(dominio, fn_loss)
hill_climb_cost     = fn_loss(solution_hill_climb)
hill_climb_cost

CPU times: total: 46.9 ms
Wall time: 44 ms


25263

In [27]:
imprimir_agenda(pessoas, solution_hill_climb)

Kevin     : SLZ/CNF - 8:04    10:11   R$ 95    |   CNF/SLZ - 15:25   16:58   R$ 62
Joao      : VCP/CNF - 13:54   18:02   R$ 294   |   CNF/VCP - 9:49    13:51   R$ 229
Dani      : POA/CNF - 20:30   23:11   R$ 114   |   CNF/POA - 6:58    9:01    R$ 238
Rafa      : FOR/CNF - 18:23   21:35   R$ 134   |   CNF/FOR - 20:27   23:42   R$ 169
Mari      : GRU/CNF - 14:22   16:32   R$ 126   |   CNF/GRU - 17:06   20:00   R$ 95
Isa       : GYN/CNF - 9:15    12:03   R$ 99    |   CNF/GYN - 16:35   18:56   R$ 144


- Outros algoritmos: PSO e o Colônia de Formigas